In [ ]:
%load_ext autoreload
%autoreload 2

# try_weighting.ipynb

import numpy as np
from datetime import datetime

from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
from adjoint_sim_sf.Optimiser import Optimiser
from adjoint_sim_sf import Experiment

# 1) COMSOL engine
if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 20,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

# (Optional) deterministic sampling for EPR sources across sweeps
evaluator.random_seed = 12345

# Default folder: exp_YYYYMMDD-HH%M-%S (comes from Experiment.__init__)
exp = Experiment()

x_vals = np.linspace(0.19, 0.21, 2 ,float)
y_vals = np.linspace(0.24, 0.26, 2,float)
param_grid = np.array([[x, y] for x in x_vals for y in y_vals], dtype=float)

# 5) Optimiser
initial_params = np.array([0.20, 0.20], float)
lr = 0.01
opt = Optimiser(initial_params=initial_params, lr=lr, evaluator=evaluator)

# 6) Sweep different JJ/EPR weightings
w_jj_list = [0.5]
timestamp = datetime.now().strftime("%Y%m%d-%H%M-%S")

# Save the run-level config once
exp.save_config({
    "adjoint_evaluator": evaluator.to_config_dict(),
    "grid": {"x_vals": x_vals.tolist(), "y_vals": y_vals.tolist()},
    "initial_params": initial_params.tolist(),
    "lr": lr,
    "w_jj_list": w_jj_list,
    "notebook_timestamp": timestamp,
})

for w_jj in w_jj_list:

    results = opt.sweep_jj_epr(param_grid, w_jj=w_jj, perturbation_mag=None, verbose=True)
    # File name encodes the weight for easy grep; JSON-serialisation handled by Experiment
    results_filename = f"sweep_jj_epr_w{w_jj:.2f}_{timestamp}.jsonl"
    exp.save_results(results, results_filename)
    print("Wrote:", exp.path(results_filename))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Commencing experiment: experiments/exp_20251019_173016
forward sim time : 16.566108
adjoint sim time : 17.607163
New gradient calculation:
[(-4.885807428167594e-05, 1.2000000000000004e-05), (-5.114006590321646e-05, 1.2065146623040944e-05), (-0.00015039628363952776, 1.773693049368731e-05), (-0.00016542996368070395, 2.161394817756866e-05), (-0.00017785500746505135, 3.092314194817185e-05), (-0.00018579956038941752, 4.426206546296618e-05), (-0.00018806676164433004, 5.962118653714587e-05), (-0.0001811221345252661, 0.00020194940266649947), (-0.00017754054329541914, 0.00021664809913075103), (-0.00016875610987956573, 0.00022896529049276027), (-0.00015602544458713874, 0.000237139006581426), (-0.0001411696648989233, 0.00024), (0.0001411696648989233, 0.00024), (0.00015602544458713874, 0.000237139006581426), (0.0001687561098795657, 0.00022896529049276027), (0.00017754054329541914, 0.000216648099130751), (0.0001

In [ ]:
from adjoint_sim_sf import Experiment, Plotter



rel_filename = "sweep_pure_epr_200_20251015-2021-52.jsonl"
data = exp.load_jsonl(rel_filename, loss_key="loss_epr", grad_key="grad_epr")
print(len(data["records"]))
print(data["records"][0])

# Compute numeric gradients
grads = exp.compute_numeric_grads(
    params_array=data["params_array"],
    loss_array=data["loss_array"],
    adj_grad_array=data["adj_grad_array"],
)

# Contour + quiver plots (only if param_dim == 2). Saves PNG in the experiment dir.
exp.plot_contours_if_2d(
    data_dict=data,
    grads_dict=grads,
    max_to_show=50,                 # 0 = show all; else ~this many arrows
    title_suffix=data["loss_key"],  # e.g. "loss_epr"
    save_name=None,                 # default: "contours_numeric_vs_adjoint.png"
)

# Magnitude comparison (+ summary stats). Saves PNG in the experiment dir.
exp.plot_magnitude_comparison(
    grads_dict=grads,
    scale=0.02,                     # apply scaling to adjoint magnitudes for the plot
    uselog=True,
    title_suffix=data["loss_key"],
    save_name=None,                 # default: "gradient_magnitude_comparison.png"

In [ ]:
# # num_sample_vary

# import numpy as np
# from datetime import datetime

# from SQDMetal.COMSOL.Model import COMSOL_Model
# from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
# from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
# from adjoint_sim_sf.Optimiser import Optimiser
# from adjoint_sim_sf import Experiment

# # 1) COMSOL engine
# if COMSOL_Model._engine is None:
#     COMSOL_Model.init_engine()

import time
import numpy as np

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 20,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

# (Optional) deterministic sampling for EPR sources across sweeps
evaluator.random_seed = 12345

# Default folder: exp_YYYYMMDD-HH%M-%S (comes from Experiment.__init__)
exp = Experiment()

x_vals = np.linspace(0.18, 0.21, 2 ,dtype=float)
y_vals = np.linspace(0.24, 0.26, 2 ,dtype=float)
param_grid = np.array([[x, y] for x in x_vals for y in y_vals], dtype=float)

# 5) Optimiser
initial_params = np.array([0.20, 0.20], float)
lr = 0.01
opt = Optimiser(initial_params=initial_params, lr=lr, evaluator=evaluator)

n_adjoint_samples_list = [10, 20, 30]
timestamp = datetime.now().strftime("%Y%m%d-%H%M-%S")

# Save the run-level config once
exp.save_config({
    "experiment_name": "Pure EPR, differing N adjoint_samples",
    "adjoint_evaluator": evaluator.to_config_dict(),
    "grid": {"x_vals": x_vals.tolist(), "y_vals": y_vals.tolist()},
    "initial_params": initial_params.tolist(),
    "lr": lr,
    "n_adjoint_samples_list": n_adjoint_samples_list,
    "notebook_timestamp": timestamp,
})


for n in n_adjoint_samples_list:
    start_time = time.time()    

    opt.evaluator.num_adj_sample_points = n
    results = opt.sweep_jj_epr(param_grid, w_jj=0, perturbation_mag=None, verbose=True)
    runtime = time.time() - start_time
    # File name encodes the weight for easy grep; JSON-serialisation handled by Experiment
    results_filename = f"sweep_pure_epr_{n}_{timestamp}.jsonl"
    exp.save_results(results, results_filename)
    print(f"Wrote: {exp.path(results_filename)}  Time Taken: {runtime:.2f}")



Commencing experiment: experiments/exp_20251019_101637
inner product time : 27.199364
sa: 10 sources, loss=4.378e+23


ValueError: too many values to unpack (expected 2)